In [1]:
import glob
print(glob.glob("/share/*"))

from torch import nn as nn
import torch
from timeseries.timeseries_transformer import TimeseriesDataloader, TimeseriesTransformerConfig, DistancePositionalEmbedding, PositionalEmbedding,TimeseriesPandasTrainer
from t3_karpathy.commons.feed_forwards import GeluFeedForward
from t3_karpathy.enhanced_karpathy_transformer import BlockSequence

from t3_karpathy.commons.commons import AbstractCodec, AbstractAccumulativeTrainer, AbstractRunner, \
    BaseTransformerConfig, TimeseriesFeedForward

print(torch.cuda.is_available())
directory_path = '/share/workspace/us-stock-dataset/Data/StockHistory'


['/share/run.sh', '/share/cuda_test.ipynb', '/share/pytorch_experiments', '/share/workspace', '/share/jupyter_notebooks']
True


In [2]:
!ls /share/workspace

Transformers1  us-stock-dataset


In [3]:

stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]


In [4]:
dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

/share/workspace/Transformers1/timeseries/csv_reader.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='bfill').reset_index(drop=True)
/share/workspace/Transformers1/timeseries/csv_reader.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='bfill').reset_index(drop=True)


Found files:  138


In [5]:
data=dataloader.get_data().transpose(0, 1).cuda()
data.shape

torch.Size([138, 5283])

In [6]:
training_data = torch.diff(data, dim=1)


In [7]:
training_data.std(dim=1)

tensor([ 0.6398,  1.2063,  1.0164,  0.5260,  0.8853,  1.1809,  0.8453,  0.7936,
         3.9333,  1.2038,  0.6027,  0.5592,  3.0230,  1.7608,  1.0342,  1.0165,
         2.3644,  2.1521,  0.5697,  1.6320,  0.5763,  1.6616,  3.4684,  0.7269,
         0.3182, 18.9003,  0.3620,  4.9890,  1.0428,  1.1672,  0.7035,  1.0692,
         3.4479,  1.6608,  1.8272,  2.8596,  2.7063,  0.5475,  5.7229,  0.6218,
         2.5938,  0.3719,  0.2764,  3.6235,  8.5105,  0.8572,  0.6603,  0.4060,
         3.7318,  1.9303,  0.5093,  1.1625,  0.3231,  2.2037,  1.1530,  1.6506,
         0.5888,  1.2393,  3.4479,  0.2984,  1.5064,  0.4031,  1.9739,  0.9988,
         2.3050,  0.7003,  1.2495,  2.4140,  1.8306,  1.6506,  0.4953,  1.2254,
         1.3827,  2.5348,  0.9783,  2.1883,  2.2940,  0.8306,  0.5245, 10.0196,
         0.7494,  1.1038,  3.0676,  0.4462,  0.5824,  4.1373,  3.5554,  1.2765,
         0.4980,  1.2125,  0.6543,  1.6320,  0.5592,  0.6601,  0.9522,  0.8087,
         0.8628,  1.1187,  0.3025,  0.58

In [8]:
def min_max_scaling(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())


In [20]:
training_data = min_max_scaling(training_data)
num_of_channels=len(training_data)

print(training_data.shape)

torch.Size([5282, 138])


In [21]:
training_data = training_data.transpose(0, 1)
print(training_data.shape)

torch.Size([138, 5282])


In [22]:
window_size = 10

def extract_sliding_windows(tensor, window_size):        
    windows = tensor.unfold(1, window_size, 1)
    return windows

sliding_windows=extract_sliding_windows(training_data, window_size).transpose(0, 1)

print(sliding_windows.shape)


torch.Size([5273, 138, 10])


In [ ]:
from t4.timeseries_transformer2 import TransformerConfig, TransformerRunner, TransformerModel
from t4.timeseries_transformer2_tokens import NoTokenTransformerRunner

config = TimeseriesTransformerConfig(
    batch_size=32,
    block_size=10,
    n_embed=32,
    n_head=4,
    n_layer=8,
    kernel_size=1,
    learning_rate=1e-3,
    channels=1
)
config.eval_interval=50
config.input_embed=num_of_channels
config.causal=True
config.vocab_size=1
config.output_num=1

trainer1 = NoTokenTransformerRunner(config, training_data, training_data)

trainer1.train_iterate_n(20000)


0.792481 M parameters
torch.Size([32, 10, 5282])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([320, 5282])) that is different to the input size (torch.Size([320, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
t

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([320, 5282])) that is different to the input size (torch.Size([320, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
torch.Size([32, 10, 5282])
t